In [8]:
from pathlib import Path

import numpy as np
import pandas as pd

from dpipe.io import load, PathLike
from pathlib import Path
import os

def choose_root(*paths: PathLike) -> Path:
    for path in paths:
        path = Path(path)
        if path.exists():
            return path
    raise FileNotFoundError('No appropriate root found.')

In [24]:
path_base_dm = choose_root(
    '/gpfs/data/gpfs0/b.shirokikh/experiments/da/spottune/wmh/baseline/wmh_dm_one2all/',
    '/nmnt/x4-hdd/experiments/da_exps/wmh/baseline/wmh_dm_one2all/',
)

path_oracle_dm = choose_root(
    '/gpfs/data/gpfs0/b.shirokikh/experiments/da/spottune/wmh/baseline/wmh_dm_oracle/',
    '/nmnt/x4-hdd/experiments/da_exps/wmh/baseline/wmh_dm_oracle/',
)

path_base_unet = choose_root(
    '/gpfs/data/gpfs0/b.shirokikh/experiments/da/spottune/wmh/baseline/wmh_unet3d_one2all/',
    '/nmnt/x4-hdd/experiments/da_exps/wmh/baseline/wmh_dm_one2all/',
)

path_oracle_unet = choose_root(
    '/gpfs/data/gpfs0/b.shirokikh/experiments/da/spottune/wmh/baseline/wmh_unet3d_oracle/',
    '/nmnt/x4-hdd/experiments/da_exps/wmh/baseline/wmh_dm_oracle/',
)

meta = choose_root(
    '/nmnt/x3-hdd/data/da_mri/wmh_ants/meta.csv',
    '/gpfs/data/gpfs0/b.shirokikh/data/wmh_ants/meta.csv',
)

path_base = path_base_unet
path_oracle = path_oracle_unet

meta = pd.read_csv(meta, index_col='id')
meta.head()

,flair,brain_mask,wmh,t1,fold,cite,x,y,z
id,,,,,,,,,
sub-Amst106,sub-Amst106/FLAIR.nii.gz,sub-Amst106/bm.nii.gz,sub-Amst106/wmh.nii.gz,sub-Amst106/T1.nii.gz,0,Amst,1.200000,0.976600,3.000000
sub-Utr23,sub-Utr23/FLAIR.nii.gz,sub-Utr23/bm.nii.gz,sub-Utr23/wmh.nii.gz,sub-Utr23/T1.nii.gz,2,Utr,0.958333,0.958333,2.999997
sub-Utr25,sub-Utr25/FLAIR.nii.gz,sub-Utr25/bm.nii.gz,sub-Utr25/wmh.nii.gz,sub-Utr25/T1.nii.gz,2,Utr,0.958333,0.958333,3.000000
sub-Sing56,sub-Sing56/FLAIR.nii.gz,sub-Sing56/bm.nii.gz,sub-Sing56/wmh.nii.gz,sub-Sing56/T1.nii.gz,1,Sing,1.000000,1.000000,2.999999
sub-Utr0,sub-Utr0/FLAIR.nii.gz,sub-Utr0/bm.nii.gz,sub-Utr0/wmh.nii.gz,sub-Utr0/T1.nii.gz,2,Utr,0.958333,0.958333,2.999999


In [25]:
records = []
for s in sorted(meta['fold'].unique()):
    res_row = {}
    
    # one2all results:
    sdices = load(path_base / f'experiment_{s}/test_metrics/dice_score.json')
    for t in sorted(set(meta['fold'].unique()) - {s}):
        df_row = meta[meta['fold'] == t].iloc[0]
        target_name = df_row['cite'] 
        
        ids_t = meta[meta['fold'] == t].index
        res_row[target_name] = np.mean([sdsc for _id, sdsc in sdices.items() if _id in ids_t])
    
    df_row = meta[meta['fold'] == s].iloc[0]
    source_name = df_row['cite']  
    sdices = {}
    for n_val in range(3):
        sdices = {**sdices,
                  **load(path_oracle / f'experiment_{s * 3 + n_val}/test_metrics/dice_score.json')}
    res_row[source_name] = np.mean(list(sdices.values()))

    res_row[' '] = source_name
    records.append(res_row)

In [26]:
df = pd.DataFrame.from_records(records, index=' ')

In [27]:
df[df.index]

,Amst,Sing,Utr
,,,
Amst,0.570099,0.648304,0.000000
Sing,0.499286,0.625136,0.000000
Utr,0.002036,0.000521,0.682836
